In [74]:
from tools import *

In [75]:
mpt = mpt_data(path = r'C:\Users\cjang.WILDCAT\Desktop\EIS_Manager\data\\', data = ['DE_49_9_30.mpt'])

In [76]:
bad_inds = [0,2,3]

In [77]:
mpt.df_raw.head()

,f,re,im,Z_mag,Z_phase,times,E_avg,I_avg,Cs/µF,Cp/µF,cycle_number,I Range,|Ewe|/V,|I|/A,(Q-Qo)/mA.h,Analog OUT/V,Analog IN 1/V,Y_re,Y_im,Y_mag,Y_phase,dq/mA.h,Unnamed: 22,w
0,1000018.60,32.982964,202.64922,205.31581,-80.755661,1.951485,-0.001807,0.003163,0.000785,0.000765,1.0,11,0.005754,0.000028,8.524873e-07,0.0,-100.06999,0.000782,0.004807,0.004871,80.755661,8.524873e-07,NaN,6.283302e+06
1,891246.69,39.379498,222.46672,225.92519,-79.961884,2.547497,-0.001273,0.003178,0.000803,0.000778,1.0,11,0.009239,0.000041,1.272279e-06,0.0,-100.11009,0.000772,0.004358,0.004426,79.961884,4.197920e-07,NaN,5.599868e+06
2,794332.69,46.115387,244.77170,249.07793,-79.330444,3.143489,-0.000828,0.003281,0.000819,0.000791,1.0,11,0.014093,0.000057,1.905502e-06,0.0,-100.06197,0.000743,0.003945,0.004015,79.330444,6.332227e-07,NaN,4.990939e+06
3,707949.81,52.711895,270.31552,275.40701,-78.965714,3.739490,-0.000418,0.003218,0.000832,0.000801,1.0,11,0.020291,0.000074,2.409300e-06,0.0,-100.04594,0.000695,0.003564,0.003631,78.965714,5.037982e-07,NaN,4.448180e+06
4,630956.81,59.499557,300.14435,305.98502,-78.787254,4.334481,-0.000027,0.003266,0.000840,0.000809,1.0,11,0.027698,0.000091,2.955232e-06,0.0,-100.07802,0.000635,0.003206,0.003268,78.787254,5.459320e-07,NaN,3.964419e+06


In [78]:
for i in bad_inds:
    mpt.df[0] = mpt.df[0].drop(i)
print(mpt.guesser())
for i in mpt.circuit_fit[0]:
        print(i.real, ", ", -i.imag)

file     fit_R        fit_Rs     fit_n         fit_Q  \
0  DE_49_9_30.mpt  0.015633  195963.69164  0.882513  4.423662e-09   

         fit_R2    fit_n2        fit_Q2    fit_n3    fit_Q3  
0  2.222692e+06  0.771996  2.044682e-07  0.949966  0.019579  
56.868590139059734 ,  274.43068730853093
76.73554323334196 ,  370.73484234205324
84.81172657933992 ,  409.83986522496804
93.74549000879837 ,  453.0597817311495
103.63887275837928 ,  500.87099708820705
114.58918041875366 ,  553.7216576798262
126.71259250556054 ,  612.1444513524787
140.1377146377089 ,  676.7236715503361
155.01563824404198 ,  748.1413932800581
171.4992078450368 ,  827.0752751247277
189.7735650930823 ,  914.3419657262251
210.02366820249637 ,  1010.737918195204
232.5196243303942 ,  1117.4404248373635
257.4848027265485 ,  1235.3742309390486
285.18926870673454 ,  1365.6500523189138
315.946310886524 ,  1509.5383549985888
350.20491605188244 ,  1668.8867903556652
388.2774150176808 ,  1844.8380321893806
430.6570579312496 ,  2039.29441

In [73]:
mpt.df[0].head()

,f,re,im,Z_mag,Z_phase,times,E_avg,I_avg,Cs/µF,Cp/µF,cycle_number,I Range,|Ewe|/V,|I|/A,(Q-Qo)/mA.h,Analog OUT/V,Analog IN 1/V,Y_re,Y_im,Y_mag,Y_phase,dq/mA.h,Unnamed: 22,w
1,891246.69,39.379498,222.46672,225.92519,-79.961884,2.547497,-0.001273,0.003178,0.000803,0.000778,1.0,11,0.009239,0.000041,0.000001,0.0,-100.11009,0.000772,0.004358,0.004426,79.961884,4.197920e-07,NaN,5.599868e+06
4,630956.81,59.499557,300.14435,305.98502,-78.787254,4.334481,-0.000027,0.003266,0.000840,0.000809,1.0,11,0.027698,0.000091,0.000003,0.0,-100.07802,0.000635,0.003206,0.003268,78.787254,5.459320e-07,NaN,3.964419e+06
5,562342.25,66.441391,333.09375,339.65558,-78.719406,4.930486,0.000333,0.003250,0.000850,0.000817,1.0,11,0.036041,0.000106,0.000004,0.0,-100.08604,0.000576,0.002887,0.002944,78.719406,5.605861e-07,NaN,3.533301e+06
6,501206.19,73.989532,369.75864,377.08871,-78.684433,5.527497,0.000635,0.003250,0.000859,0.000826,1.0,11,0.045215,0.000120,0.000004,0.0,-100.07802,0.000520,0.002600,0.002652,78.684433,8.018593e-07,NaN,3.149171e+06
7,446685.72,82.182106,410.15189,418.30429,-78.669685,6.123493,0.000882,0.003234,0.000869,0.000835,1.0,11,0.055154,0.000132,0.000005,0.0,-100.09406,0.000470,0.002344,0.002391,78.669685,2.610770e-07,NaN,2.806609e+06


In [64]:
mpt.guesser()

,file,fit_R,fit_Rs,fit_n,fit_Q,fit_R2,fit_n2,fit_Q2,fit_n3,fit_Q3
0,DE_49_9_30.mpt,0.015633,195963.69164,0.882513,4.423662e-09,2.222692e+06,0.771996,2.044682e-07,0.949966,0.019579


In [67]:
mpt